In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import welly as wl
from welly import Project 
from welly import Well
import seaborn as sns

In [ ]:
path_data = 'C:/Users/Bruger/iCloudDrive/MLO_documents/projects/LOCRETA_2018-2022/LOCRETA_DATA/'
path_wells = path_data + 'wells/all_wells/df/'
las_list = []
for file in os.listdir(path_wells):
    if file.endswith(".las"):
        print(os.path.join(path_wells, file))
        las_list.append(file)


In [ ]:
p = Project.from_las(path_wells+"*.las", index = 'm')
well_list = []
for i in range (len(las_list)):
    p[i].name = p[i].uwi
    well_list.append(p[i].uwi)

In [ ]:
well_list

In [ ]:
from IPython.display import HTML

html1= p.curve_table_html()
HTML(html1)

In [ ]:
alias={
    'DEPTH' : ['MD'],
    'GR':['GRN_C'],
    'DTC' : ['DTC_C'],
    'DTS' : ['DTS_C'],
    'DENS':['DENS_C'],
    'CALI':['CALI','DCAL','CAL','CALW'],
    'NEUT':['NEUT_C'],
    'RMED' : ['RMED_C'],
    'BIT' : ['BIT','BS']
}

keys=[]
for k in alias.keys():
    keys.append(k)

html = p.curve_table_html(keys=keys,alias=alias)

HTML(html)

In [ ]:
df = p.df(keys=keys,alias=alias)
df.reset_index(inplace=True)

df.drop(columns = 'DEPTH', inplace=True)
df.rename(columns = {'DEPT' : 'DEPTH'}, inplace=True)
df['DEPTH'].loc[df['UWI'] == 'VALDEMAR-2H'] = df['DEPTH'].loc[df['UWI'] == 'VALDEMAR-2H']*.3048

df['DTC'].loc[df['UWI'] == 'BO-3X'] =df['DTC'].loc[df['UWI'] == 'BO-3X']*.3048
df['DTS'].loc[df['UWI'] == 'BO-3X'] = df['DTC'].loc[df['UWI'] == 'BO-3X']
df['NEUT'].loc[df['UWI'] == 'BO-3X'] = df['NEUT'].loc[df['UWI'] == 'BO-3X']/100.

df.dropna(axis=0, inplace=True)

features = ['DEPTH', 'GR', 'DTC', 'DTS', 'DENS', 'CALI', 'NEUT', 'RMED', 'BIT']
for feature in features:
    df.drop(index = df.loc[df[feature]<0].index, inplace=True)

In [ ]:
zones = pd.read_excel(path_data +'Zones-wells-LOCRETA_UpperandLowerCretaceous.xlsx', names = ['uwi', 'dataset', 'MD', 'formation'])
#zones = zones.drop(index=0)
zones['MD'] = zones['MD']*0.3048 #ft to m
#print(zones.dtypes)
#zones.dropna(axis=1, inplace= True)
#zones['WellName'].unique()

In [ ]:
zones

In [ ]:
#df['DEPTH'] = df['DEPTH'] * 0.3048

In [ ]:
df

In [ ]:
sns.histplot(df, x='DEPTH', kde=True, hue = 'UWI', multiple="stack", element="step")

In [ ]:
def make_well_tops_df(df, zones, uwi):
    tops_df = zones.loc[zones['uwi']==uwi]
    tops_df['tops'] = tops_df.MD.copy()
    td = df.loc[df['UWI']==uwi]['DEPTH'].max()
    bottoms = list(tops_df['tops'][1:])
    bottoms.append(td)
    tops_df['bottoms'] = bottoms
    return tops_df

def get_formation(row, tops_df):
    md = row['DEPTH']
    if md < tops_df['tops'].min():
        return np.nan#'None' 
    condition = (tops_df['bottoms'] >= md) & (tops_df['tops'] <= md)
    #print(tops_df[condition]['ZONE_NAME'])
    name = tops_df.loc[condition]['formation'].values[0]
    #print(type(name))
    try: 
        return str(name)
    except:
        return np.nan#str('None')

In [ ]:
df['formation'] = np.nan
 
for uwi in df['UWI'].unique():
    print(uwi)
    tops_df = make_well_tops_df(df, zones, uwi)
    well_df = df.loc[df['UWI']==uwi]
    df.loc[df['UWI']==uwi, 'formation'] = well_df.apply(get_formation, axis=1, tops_df=tops_df)

In [ ]:
df.dropna(axis=0, inplace = True)

In [ ]:
df.loc[df['UWI'] == 'BO-2X']['formation']

In [ ]:
df.groupby('formation')['UWI']

In [ ]:
df.groupby('formation').count().plot(kind='bar')

In [ ]:
df.shape

In [ ]:
sns.pairplot(df[['DEPTH', 'GR', 'DTC', 'DTS', 'DENS','NEUT', 'RMED', 'UWI']].sample(1000), hue='UWI')